In [1]:
from Linear_KF import KalmanFilter_nti, KalmanFilter_NE
from RTS_Smoother import RTS_Smoother_nti, RTS_Smoother_NE
import torch
from Linear_sysmdl import SystemModel, System_Model_nti, System_Model_NE
import torch.nn as nn
from RTS_NUV import RTS_NUV, RTS_UV_nti, RTS_Simple_NUV


Running on the CPU


In [2]:
# nu = 0 dB
# r2 = 10 dB
idx_traj = 100
Y = torch.load('Sim_baseline/traj/Y_obs.pt')[0, 0, idx_traj, :, :]
X = torch.load('Sim_baseline/traj/X_gt.pt')[0, 0, idx_traj, :, :]

In [3]:
T = 100
nu_dB = 0.0
prec_r_dB = -10.0

F = torch.tensor([[1.0, 1.0],[0.0, 1.0]])
H = torch.eye(2)

F_nti = F.unsqueeze(2).repeat(1, 1, T)
H_nti = H.unsqueeze(2).repeat(1, 1, T)

r, q = pow(10, -prec_r_dB/20), pow(10, (nu_dB-prec_r_dB)/20)

Q = q*q*torch.eye(2).unsqueeze(2).repeat(1, 1, T)
R = r*r*torch.eye(2).unsqueeze(2).repeat(1, 1, T)





In [4]:
# fixed initial condition
m1x_0 = torch.tensor([1.0, 0.0])
m2x_0 = torch.zeros(size=[2, 2])

model_ne = System_Model_NE(F, H, T, Q, R)
model_ne.init_sequence(m1x_0, m2x_0)

kf_ne = KalmanFilter_NE(model_ne)
kf_ne.init_sequence(m1x_0, m2x_0)

kf_ne.generate_sequence(Y, T)
x_kf_ne = kf_ne.x
sigma_kf_ne = kf_ne.sigma

ks_ne = RTS_Smoother_NE(model_ne)
ks_ne.generate_sequence_cross(x_kf_ne, sigma_kf_ne, kf_ne.KG, Q, T)

In [5]:
model_nti = System_Model_nti(F_nti, H_nti, T, Q, R)
model_nti.init_sequence(m1x_0, m2x_0)

kf_nti = KalmanFilter_nti(model_nti)
kf_nti.init_sequence(model_nti.m1x_0, model_nti.m2x_0)

kf_nti.generate_sequence(Y, T)
x_kf_nti = kf_nti.x

ks_nti = RTS_Smoother_nti(model_nti)
ks_nti.generate_sequence_cross(kf_nti.x, kf_nti.sigma, Q, T, cross_x1x0=True, KG_end=kf_nti.KG)


In [6]:
loss_func = nn.MSELoss(reduction='mean')
mse = 10.0*torch.log(loss_func(ks_ne.s_x, ks_nti.s_x))

print(mse)


tensor(-inf)


In [7]:
torch.equal(ks_ne.sigma_cross, ks_nti.sigma_cross)

True

In [8]:
x_kf_ref = torch.load('Sim_baseline/KF/X_kf.pt')[0, 0, idx_traj, :, :]
mse_ref = 10.0*torch.log(loss_func(x_kf_ref, X))
print(mse_ref)

tensor(17.1979)


# test EM+MA

In [9]:
model = SystemModel(F, q, H, r, T, T)
model.InitSequence(m1x_0, m2x_0)
uv_r = RTS_Simple_NUV(model)
uv_r.smooth_unknownQ(Y, 20, 1.0)

In [10]:
mse_nti = loss_func(uv_r.RTS.s_x, X)
print(10*torch.log10(mse_nti))

tensor(6.8023)


In [32]:
uv_r_nti = RTS_UV_nti(model_nti)
uv_r_nti.init_KF(m1x_0, m2x_0)
uv_r_nti.smooth_unknownQ(Y, 40, 1.0, win=40)


In [33]:
mse_nti = loss_func(uv_r_nti.RTS.s_x, X)
print(10*torch.log10(mse_nti))

tensor(5.8726)


In [34]:
x_ref = torch.load('Sim_const_Q/X_unknownQ.pt')[0,0,idx_traj,:,:]
mse_ref = loss_func(x_ref, X)
print(10*torch.log10(mse_ref))

tensor(5.4453)
